In [2]:
from llama_index.evaluation import generate_question_context_pairs
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.node_parser import SimpleNodeParser
from llama_index.evaluation import generate_question_context_pairs
from llama_index.evaluation import RetrieverEvaluator
from llama_index.llms import OpenAI

import os
import pandas as pd

In [1]:
from llama_index import download_loader
from llama_index.readers.schema import Document

# Converts a single record from the Actor's resulting dataset to the LlamaIndex format
def tranform_dataset_item(item):
    return Document(
        text=item.get("text"),
        extra_info={
            "url": item.get("url"),
        },
    )

ApifyActor = download_loader("ApifyActor")

reader = ApifyActor("apify_api_kHubBvK****************eIIpaNDEp0hOjHi")

In [3]:
documents = reader.load_data(
    actor_id="apify/website-content-crawler",
    run_input={"startUrls": [{"url": "https://trigger.dev/docs/documentation"}, {"url": "https://trigger.dev/docs/integrations"}, {"url": "https://trigger.dev/docs/sdk"}, {"url": "https://trigger.dev/apis"}]},
    dataset_mapping_function=tranform_dataset_item,
)

In [8]:
os.environ['OPENAI_API_KEY'] = '************************************'
llm = OpenAI(model="gpt-3.5-turbo-1106")

In [9]:
node_parser = SimpleNodeParser.from_defaults()
nodes = node_parser.get_nodes_from_documents(documents)
vector_index = VectorStoreIndex(nodes)

In [10]:
vector_index.storage_context.persist(persist_dir="TriggerIO")